In [2]:
import { TextLoader } from 'langchain/document_loaders/fs/text';
import { RecursiveCharacterTextSplitter } from 'langchain/text_splitter';

const loader = new TextLoader('data/qiu.txt');
const docs = await loader.load();

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 100,
  chunkOverlap: 20,
});

const splitDocs = await splitter.splitDocuments(docs);

import { OllamaEmbeddings } from '@langchain/community/embeddings/ollama';
const embeddings = new OllamaEmbeddings({
  model: 'llama3.1', // default value
  baseUrl: 'http://localhost:11434', // default value
  requestOptions: {
    useMMap: true, // use_mmap 1
    numThread: 10, // num_thread 10
    numGpu: 1, // num_gpu 1
  },
});


import { Ollama } from "@langchain/ollama";
const chatModel = new Ollama({
  baseUrl: "http://localhost:11434",
  model: "llama3.1",
});


import { MemoryVectorStore } from 'langchain/vectorstores/memory';

const vectorstore = new MemoryVectorStore(embeddings);
await vectorstore.addDocuments(splitDocs);

const retriever = vectorstore.asRetriever(2);

// const result = await retriever.invoke("什么是基础设施")
// console.log('result :', result);

import { ChatPromptTemplate } from '@langchain/core/prompts';

const TEMPLATE = `
你是一个本次事故的分析者，根据文章详细解释和回答问题，你在回答时会引用作品原文。
并且回答时仅根据原文，尽可能回答用户问题，如果原文中没有相关内容，你可以回答“原文中没有相关内容”，

以下是原文中跟用户回答相关的内容：
{context}

现在，你需要基于原文，回答以下问题：
{question}`;

const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

import { RunnableSequence } from '@langchain/core/runnables';
import { Document } from '@langchain/core/documents';

const convertDocsToString = (documents: Document[]): string => {
  return documents.map((document) => document.pageContent).join('\n');
};

const contextRetriverChain = RunnableSequence.from([
  (input) => input.question,
  retriever,
  convertDocsToString,
]);

const ragChain = RunnableSequence.from([
  {
    context: contextRetriverChain,
    question: (input) => input.question,
  },
  prompt,
  chatModel,
]);

console.log('开始调用', ragChain);

const answer = await ragChain.invoke({
  question: '什么是基础设施',
});

console.log(answer);


开始调用 RunnableSequence {
  lc_serializable: true,
  lc_kwargs: {
    first: RunnableMap {
      lc_serializable: true,
      lc_kwargs: {
        steps: { context: [RunnableSequence], question: [RunnableLambda] }
      },
      lc_runnable: true,
      name: undefined,
      lc_namespace: [ "langchain_core", "runnables" ],
      steps: {
        context: RunnableSequence {
          lc_serializable: true,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: undefined,
          first: [RunnableLambda],
          middle: [Array],
          last: [RunnableLambda],
          lc_namespace: [Array]
        },
        question: RunnableLambda {
          lc_serializable: false,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: undefined,
          lc_namespace: [Array],
          func: [Function: question]
        }
      }
    },
    middle: [
      ChatPromptTemplate {
        lc_serializable: true,
        lc_kwargs: {
          inputVariabl